In [1]:
%cd C:\Users\yukir\Documents\GitHub\Scrapper\py

from nlp_processing import *
set_global_determinism(seed=1337)

%cd C:\Users\yukir\Documents\Monicas_workspace\Korea_medicine\LDA

C:\Users\yukir\Documents\GitHub\Scrapper\py
True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9039370264300117154
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9712120788964023219
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9883535296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6811741475758386365
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15475955615230952901
physical_device_desc: "device: XLA_GPU device"
]
tf 2.2.0
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
C:\Users\yukir\Documents\Monicas_workspace\Korea_medicine\LDA


https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/

In [2]:
from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
import pyLDAvis

from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from konlpy.tag import Mecab 
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm 
import re 
import pickle 
import csv

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from gensim.models import CoherenceModel
import gensim
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


In [3]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 
    
def get_nouns(tokenizer, sentence): 
    """ 단어의 길이가 2이상인 일반명사(NNG),
     고유명사(NNP), 외국어(SL)만을 반환한다. 
     :param tokenizer: 
     :param sentence: :return: """ 
    tagged = tokenizer.nouns(sentence)
    nouns = [s for s in tagged if len(s)>1] 
     
    return nouns 
     
     
def tokenize(df): 
    tokenizer = okt
    processed_data = [] 
    for sent in tqdm(df['cleaned_review']):
        sentence = clean_text(sent.replace('\n', '').strip()) 
        processed_data.append(get_nouns(tokenizer, sentence)) 
        
    return processed_data
    
    
def save_processed_data(processed_data): 
    """ 토큰 분리한 데이터를 csv로 저장 :param processed_data: :return: """ 
    
    with open('lsa_token_pos.csv', 'w', newline='', encoding='utf-8') as f: 
        writer = csv.writer(f) 
        for data in processed_data:
             writer.writerow(data)


In [4]:
cleaned_nlp = pd.read_csv('cleaned_NLP.csv', index_col =0)
cleaned_nlp.head()
cleaned_nlp = cleaned_nlp.drop_duplicates()
cleaned_nlp = cleaned_nlp.dropna()
df = cleaned_nlp.copy()
df = df[df['stars']>6.2]
df.head(3)

,hospital,stars,reviews,adress,cleaned_review
1,,,,,
0,청주나비솔한의원,10.0,병원이 전반적으로 깔끔해서 쾌적했어요대기가 좀 있지만 심하진 않은편이었고 간호사분들...,충북,병원이 전반적으로 깔끔해서 쾌적했어요 대기가 좀 있지만 심하진 않은 편이었고 간호사...
1,청주나비솔한의원,7.5,체인 한의원이라서 망설였는데 시설도 깔끔하고 진료도 친절하게 너무 잘 봐주셨어요 그...,충북,체인 한의원이라서 망설였는데 시설도 깔끔하고 진료도 친절하게 너무 잘 봐주셨어요 그...
2,청주나비솔한의원,10.0,사고 후유증으로 놀라서 한의원 방문 하였고 침 놔주시고 청심환 처방 해주셨습니다 편...,충북,사고 후유증으로 놀라서 한의원 방문하였고 침 놔주시고 청심환 처방해주셨습니다 편안하...


In [5]:
if __name__ == '__main__':
     # df = pd.read_csv('negative.csv', index_col = 0)
     processed_data = tokenize(df) # description 부분을 토크나이징 한다. 
     
     # 토큰 분리한 데이터를 저장 
     save_processed_data(processed_data)



100%|██████████| 14410/14410 [02:49<00:00, 84.81it/s]


In [6]:
from nltk.corpus import stopwords  
stopwords.words('korean')[:10] 

stopwords_kor = stopwords.words('korean')

## **TF-IDF행렬 만들기**

In [7]:
detokenized_doc = []
for i in range(len(processed_data)):
    t = ' '.join(processed_data[i])
    detokenized_doc.append(t)

df['detokenized_doc'] = detokenized_doc # 다시 text['headline_text']에 재저장

df['detokenized_doc'][:5]

1
0                   병원 전반 대기 간호사 의사 설명 주말 공휴일 진료 가기 항상
1                체인 한의원 시설 진료 양심 진료 믿음 무조건 한약 치료 모습 믿음
2               사고 후유증 한의원 방문 청심환 처방 휴식 의원 내부 다음 기회 방문
3                          발바닥 방문 진료 별로 효과 계속 대체 문제 계속
4    다이어트 처방 효과 처음 체질 진단 체질 평소 습관 운동 상태 확인 처방 정말 진료...
Name: detokenized_doc, dtype: object

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words=stopwords_kor,             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[가-힣0-9]{1,}',  # num chars > 3
                             max_features=2000,             # max number of uniq words
                            ) 
data_vectorized = vectorizer.fit_transform(df['detokenized_doc'])
data_vectorized.shape # TF-IDF 행렬의 크기 확인

(14410, 1716)

In [9]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.8511312883276797 %


In [19]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=6,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=1337,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_top=lda_model.fit_transform(data_vectorized)

print(lda_model.components_)
print(lda_model.components_.shape) 

[[  0.16771378   0.17015908   0.167965   ...   0.16673506   0.16788923
    3.92142421]
 [  0.1678575    0.1679143    0.16773139 ...  33.15045407   0.16732442
    0.16669701]
 [ 69.44681189 145.62698826  40.31891433 ...   0.16707748   0.1682989
    0.16821777]
 [ 46.56474578  42.13588368  23.85236    ...   0.16682507  21.83601802
    0.16849391]
 [  1.06355816   7.1545283    0.16780575 ...   0.16745022   0.16835288
    0.16748756]
 [  0.1676623    0.16786826   0.17049789 ...   0.16672812   0.16755405
    0.17047146]]
(6, 1716)


In [11]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
print(lda_model.get_params())

Log Likelihood:  -369694.16433012555
Perplexity:  1792.5667125740702
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 7, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 1337, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


## **GRID SEARCH**

In [ ]:
# Define Search Param
search_params = {'n_components': [3, 5, 7, 10], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 3}
Best Log Likelihood Score:  -1917.6329268049885
Model Perplexity:  282.34220413185625


## **OUTPUT**

In [20]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(df['cleaned_review'].tolist()))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.040000,0.040000,0.660000,0.040000,0.040000,0.180000,2
Doc1,0.040000,0.040000,0.330000,0.040000,0.500000,0.040000,4
Doc2,0.040000,0.040000,0.780000,0.040000,0.040000,0.040000,2
Doc3,0.050000,0.050000,0.540000,0.050000,0.050000,0.250000,2
Doc4,0.040000,0.040000,0.040000,0.040000,0.810000,0.040000,4
Doc5,0.030000,0.030000,0.830000,0.040000,0.030000,0.030000,2
Doc6,0.050000,0.050000,0.050000,0.050000,0.730000,0.050000,4
Doc7,0.040000,0.040000,0.820000,0.040000,0.040000,0.040000,2
Doc8,0.080000,0.030000,0.110000,0.660000,0.080000,0.030000,3
Doc9,0.040000,0.040000,0.040000,0.790000,0.040000,0.040000,3


In [21]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('보약', 72.15), ('전문', 60.62), ('한결', 52.47), ('손님', 42.07), ('양방', 36.37), ('산후', 34.64), ('손가락', 32.9), ('오시', 32.05), ('엄마', 31.69), ('골반', 29.45)]
Topic 2: [('감기', 46.65), ('흉터', 33.15), ('결제', 31.88), ('기구', 30.35), ('상가', 27.16), ('기운', 24.76), ('피로', 23.2), ('질병', 23.02), ('멀리', 22.91), ('기대', 22.06)]
Topic 3: [('치료', 615.48), ('진료', 555.04), ('병원', 471.84), ('선생님', 459.43), ('방문', 437.42), ('원장', 413.56), ('한의원', 398.47), ('설명', 364.52), ('직원', 330.69), ('의사', 328.24)]
Topic 4: [('치료', 392.87), ('방문', 300.32), ('진료', 294.16), ('허리', 275.98), ('선생님', 268.38), ('통증', 240.87), ('물리치료', 240.17), ('병원', 219.56), ('어깨', 204.12), ('원장', 194.24)]
Topic 5: [('체질', 137.86), ('한약', 133.39), ('다이어트', 88.43), ('검사', 87.66), ('진맥', 83.28), ('확인', 80.68), ('처방', 59.14), ('상담', 56.07), ('복용', 50.94), ('곳도', 46.73)]
Topic 6: [('입원', 82.26), ('리뷰', 39.39), ('오픈', 37.14), ('컨디션', 31.48), ('무척', 29.9), ('인대', 28.66), ('처리', 28.3), ('나니', 27.3), ('염증', 27.23), ('교통사고', 26.88)]


In [22]:
# dictionary = corpora.Dictionary(processed_data) 
# dictionary.filter_extremes(no_below=10, no_above=0.05) 
# corpus = [dictionary.doc2bow(text) for text in processed_data] 

In [23]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\sklearn\manifold\_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2       1.465019 -47.625065       1        1  46.004263
3      54.854797 -36.476795       2        1  23.392782
4      32.212929 -61.444572       3        1  11.271888
5       5.111096 -14.110332       4        1   7.028056
0      26.350170 -33.374702       5        1   6.810239
1      38.109818  -7.222781       6        1   5.492772, topic_info=     Term        Freq       Total Category  logprob  loglift
1622   한약  287.000000  287.000000  Default  30.0000  30.0000
1449   체질  183.000000  183.000000  Default  29.0000  29.0000
1171   입원  119.000000  119.000000  Default  28.0000  28.0000
572    보약  108.000000  108.000000  Default  27.0000  27.0000
1652   허리  420.000000  420.000000  Default  26.0000  26.0000
...   ...         ...         ...      ...      ...      ...
1300   조심   22.163088   23.125971   Topic6  -4.8065   2.8592
1269   정기   22.095104   23.057236   Topic6  -4.8096   2.8591
543    번의   21.783474   22.748479   Topic6  -4.8238   2.8584
1401   질환   24.585080   39.299656   Topic6  -4.7028   2.4327
317    당일   22.431177   34.741558   Topic6  -4.7945   2.4643

[333 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1         1  0.720590   가격
1         2  0.226301   가격
1         3  0.047642   가격
9         3  0.419298   가도
9         4  0.419298   가도
...     ...       ...  ...
1699      4  0.007279   효과
1699      5  0.002426   효과
1702      2  0.951729   후기
1713      6  0.982196   흉터
1714      2  0.955478   힐링

[445 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 6, 1, 2])

In [24]:
df_document_topic.to_csv('LDA_tfidf_pos.csv')

In [25]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,2,9299
1,3,4166
2,4,750
3,5,95
4,0,72
5,1,28


In [27]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,가게,가격,가기,가까이,가끔,가나,가능,가능성,가다가,가도,가라,가량,가려움증,가루,가면,가보,가성,가세,가스,가슴,가시,가야,가요,가운,가장,가정,가족,각오,간격,간다,간단,간식,간이,간판,간호,간호사,간호조무사,간혹,갈수록,감기,감동,감량,감사,감소,감수,감안,갑자기,강남,강남역,강도,...,호불호,호전,호텔,호흡,호흡기,홍보,홍조,홍채,화가,화로,화면,화병,화장,화장실,화제,화차,확신,확인,확장,환경,환기,환도,환부,환불,환약,환자,환절기,활동,활용,회복,회사,회전,횟수,효과,효능,효율,후기,후다닥,후보,후원,후유증,후의,후회,휴가,휴무,휴식,휴일,흉터,힐링,힘줄
Topic0,0.167714,0.170159,0.167965,0.167612,3.434915,0.167169,0.168258,5.662979,6.278420,6.538950,0.167028,0.168570,0.167221,0.167366,0.168004,0.168227,0.167446,0.168487,0.167135,0.172335,0.168550,0.167738,0.167941,0.167685,0.167720,4.988554,0.169420,0.169546,0.167250,0.167158,0.166843,0.167026,6.035488,0.167469,0.168385,0.177518,0.168120,0.168809,0.167569,0.167146,0.168035,0.167412,0.167736,0.166939,0.167394,0.167507,11.152262,0.168023,0.167055,0.167310,...,4.670530,0.168388,0.168105,0.170942,0.167400,0.168750,0.167677,0.167721,0.166866,0.167300,0.166879,4.487261,0.167602,0.167567,0.167367,0.167206,0.167421,0.167887,0.167346,0.167176,0.167090,0.166933,0.167123,0.167668,0.167013,0.170694,0.167660,0.172880,0.167453,5.289715,0.173274,0.167014,0.167404,0.843717,0.167529,0.167058,0.171735,0.167163,0.169412,0.167700,0.167528,0.166957,0.167150,0.166896,0.170471,0.167361,0.166864,0.166735,0.167889,3.921424
Topic1,0.167857,0.167914,0.167731,0.168077,0.167192,0.168806,0.167472,0.167385,0.166911,0.167725,14.271684,0.167377,4.947326,0.167702,0.167580,0.167220,0.167587,0.167226,0.167324,0.168949,0.169341,0.167650,0.167826,0.166768,0.167577,0.166849,0.168863,0.166725,0.166804,17.063522,0.166687,0.166842,0.167647,0.166983,0.167199,0.167552,0.169570,0.167004,0.167168,46.647198,0.167518,0.167219,0.167287,0.166888,0.168230,0.167235,0.168045,0.167547,0.166779,0.167455,...,0.167100,0.167684,0.167006,0.168951,0.170051,0.176244,0.167457,0.167473,0.167868,0.166813,0.167401,0.167942,3.141106,0.167656,12.560376,0.167866,0.168317,0.167887,0.167789,0.168282,0.167217,0.167753,5.334178,0.168184,0.167570,0.167544,5.045319,0.166846,5.793253,0.168918,0.167951,0.166998,0.168857,0.167998,0.167265,0.167187,0.170256,0.167277,0.167304,0.167053,0.167585,0.167088,7.226526,0.167708,0.168812,0.169176,0.166938,33.150454,0.167324,0.166697
Topic2,69.446812,145.626988,40.318914,25.924761,61.374758,3.872694,0.168326,0.168616,0.167495,0.170090,0.167863,12.253586,0.167431,0.168565,83.607526,7.085799,0.168273,18.228991,0.168023,0.172188,0.169342,58.408180,29.699501,0.167861,41.821443,0.167537,0.169854,3.957757,7.307123,0.167969,0.167483,7.815874,0.167874,0.167887,20.248783,230.136021,0.169257,6.627698,0.167971,0.167611,24.971162,30.021780,0.167758,0.167409,5.901027,11.084625,0.169547,0.169248,4.961586,16.072280,...,0.168658,80.712960,0.168296,0.169790,6.052546,0.169388,4.283164,0.167316,0.167623,0.167444,0.167913,0.167997,0.167259,28.833236,0.167340,0.168441,0.168926,0.170037,0.168362,37.939703,0.169476,0.168155,0.167536,5.381914,6.341363,213.209042,0.167923,4.700079,0.167660,28.250342,42.631701,0.167571,5.645620,305.859106,0.168483,4.744710,0.171849,0.166972,0.167215,6.280088,28.650430,3.489554,0.167745,6.662609,0.168809,8.522107,0.167438,0.167077,0.168299,0.168218
Topic3,46.564746,42.135884,23.852360,0.168674,0.171466,0.168760,10.967681,0.166945,0.167995,0.168996,0.167441,0.168520,0.166869,0.167167,30.332520,0.167938,0.168044,0.167739,0.167276,10.686310,21.644462,0.169032,6.670921,0.168514,19.100613,0.167673,40.826727,0.166822,0.167107,0.167473,0.167966,0.167355,0.167495,0.167462,0.168593,149.160209,0.168102,0.167829,0.167198,0.167097,0.168593,0.166874,0.167431,0.168937,0.167506,3.073468,58.376785,14.906671,0.167646,0.168447,...,0.167606,61.426847,0.167808,0.167473,0.166941,8.406063,0.166797,0.166760,3.736336,0.168497,0.167905,0.168319,0.167912,0.168164,0.167393,0.167240,0.167165,0.169205,7.033055,0.169251,0.166888,0.166827,0.168002,0.167155,0.167388,46.055009,0.166979,0.173005,0.167109,14.026470,44.755089,0.168226,0.173687,115.144656,10.922980,1.263360,33.489225,3.709446,0.166961,0.168148,0.168261,0.167466,0.167196,0.168284,7.716012,0.169362,11.592391,0.1

In [29]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,보약,전문,한결,손님,양방,산후,손가락,오시,엄마,골반,어머니,대화,염좌,강추,위주
Topic 1,감기,흉터,결제,기구,상가,기운,피로,질병,멀리,기대,안정,면역,생기,아파트,요새
Topic 2,치료,진료,병원,선생님,방문,원장,한의원,설명,직원,의사,효과,추천,물리치료,정말,의원
Topic 3,치료,방문,진료,허리,선생님,통증,물리치료,병원,어깨,원장,설명,추나,의사,한의원,직원
Topic 4,체질,한약,다이어트,검사,진맥,확인,처방,상담,복용,곳도,중간,효과,음식,방문,통해
Topic 5,입원,리뷰,오픈,컨디션,무척,인대,처리,나니,염증,교통사고,이름,수술,개원,보험,나은
